# Tutorial for Toy Generation  to be used for limit production

#### Some intro here...  //FIXME TODO

First of all I suggest you to produce a single compilable macro that contains the likelihood you want to use, you can find in other tutorial how to define the likelihood (LINK HERE //TODO). The idea of this macro is to define a function that you can use to retrieve the likelihood, so that you have it defined in one place only. You can get inspiration on how to write this function from here (